# Simulation command generation from simulation sheet
>'It's turtles all the way down!'

Plan: 
- Have array of sample settings, maybe optimized to give comparable $\sigma t$ to give comparable signal strength.

In [ ]:
from definitions import s_t
import numpy as np
import pandas as pd
import os 


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 3e-3), (50e-9, 10e-3)]
# Source (L0, DL) [(m,m)]
sources = [(2.165, 0.02165), (4.321, 0.04321), (8, 0.8)]
# Universal sample parameters (?)
phi = 0.02

delta_rho = 2e14 # 1/m^2 (?)
for (R, t) in samples:
    print(F"R = {R * 1e9}nm, t = {t}m:")
    for (L0, DL) in sources:
        st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
        permissible = st < 0.8 and st >= 0.1
        print(f"\t lambda = {L0} Å: sigma * t: {st}, permissible: {permissible}")
        sigma = s_t(R,t, L0 * 1e-10) / t
        t_optimum = 0.45 / sigma
        print(f"\t Optimal t: t={t_optimum}")
        # print(s_t(R,t_optimum, L0 * 1e-10))

In [ ]:
from definitions import s_t


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]
# Universal sample parameters (?)
phi = 0.015

delta_rho = 2.08e14 # 1/m^2 (?)
for (R, t) in samples:
    print(F"R = {R * 1e9}nm, t = {t}m:")
    for (L0, DL) in sources:
        st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
        permissible = st < 0.8 and st >= 0.1
        print(f"\t lambda = {L0} Å: ")
        print(f"\t\ts*t: {st}\n\t\t0.1 <= s*t <= 0.8: {permissible}")
        sigma = s_t(R,t, L0 * 1e-10) / t
        t_optimum = 0.45 / sigma
        # print(f"\t Optimal t: t={t_optimum}")
        # print(s_t(R,t_optimum, L0 * 1e-10))

In [ ]:
print("{:,.3e}".format(np.sqrt(6.1e-7) * 1e18))

In [ ]:
from definitions import *


# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
Rs = [2e-6, 300e-9, 50e-9]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]

# Source sample (thickness) pairing (L0, DL, R, t)
source_sample = [(4.321, 0.04321, 2e-6, 1e-3), (8, 0.8, 2e-6, 1e-3), (4.321, 0.04321, 300e-9, 10e-3), (8, 0.8, 300e-9, 5e-3), (4.321, 0.04321, 50e-9, 10e-3), (8, 0.8,50e-9, 10e-3)]

thickness_map = {
    (4.321, 2e-6):  1e-3,
    (8, 2e-6):  1e-3,
    (4.321, 300e-9): 10e-3,
    (8, 300e-9): 5e-3,
    (4.321, 50e-9): 10e-3,
    (8, 50e-9): 10e-3,
}
# Universal sample parameters (?)
phi = 0.015

delta_rho = 1.8e14 # 1/m^2 (?)
print(f"Constant parameters: drho = {delta_rho * 1e-14}e14 m^-2; phi = {phi}")
for (L0, DL, R, t) in source_sample:
    # print(thickness_map[(L0, R)])
    print(F"R = {R * 1e9}nm, t = {round(t * 1e3,2)}mm, lambda = {L0} Å:")
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    permissible = st < 0.8 and st >= 0.1
    print(f"\ts*t: {st}\n\t0.1 <= s*t <= 0.8: {permissible}")
    sigma = s_t(R,t, L0 * 1e-10) / t
    t_optimum = 0.38 / sigma

In [ ]:
import pandas as pd
import os 
df = pd.read_csv('simulations_new.csv', sep=',', header=0)

FWHM_env_min = 0.005
class Instrument:
    def __init__(self, id: str,  prec_type: str, L0: float, DL: float, theta_0: float, By_min: float, By_max: float, L_s: float = 1.8):
        self.id = id
        self.prec_type = prec_type
        self.L0 = L0
        self.DL = DL
        self.theta_0 = theta_0
        self.By_min = By_min 
        self.By_max = By_max
        self.L_s = L_s

    def delta_min(self):
        return compute_z(self.By_min,self.theta_0,self.L0,self.L_s)
    
    def delta_max_named(self):
        d_max_field = self.delta_max_B_field()
        delta_max_env = self.delta_max_envelope()
        delta_max_ten_samples = self.delta_max_sampling()
        maxes = [(d_max_field, 'B strength'), (delta_max_env, 'envelope'), (delta_max_ten_samples, 'sampling')]
        (d_max, max_name) = min(maxes)
        return (d_max, max_name) 

    def delta_max(self):
        return self.delta_max_named()[0]
    
    def delta_max_B_field(self):
        # Due to the focussing condition, one component will have field By_max/2 and the other By_max, giving a delta of By_max/2
        return compute_z(self.By_max/2,self.theta_0,self.L0,self.L_s)

    def delta_max_sampling(self, samples = 10):
        f_s = 1/detector_pixel_size
        f_super_sampled = f_s / samples
        delta_max_sampling = f_super_sampled * self.L0 * self.L_s
        return delta_max_sampling

    def delta_max_envelope(self):
        delta_max_env = np.sqrt(2 * np.log(2)) * self.L0**2 * self.L_s / (np.pi * self.DL * FWHM_env_min)
        return delta_max_env
    
    def delta_range(self):
        return self.delta_min(), self.delta_max()

    def __str__(self):
        d_min = self.delta_min()
        d_max_field = self.delta_max_B_field()
        delta_max_env = self.delta_max_envelope()
        delta_max_ten_samples = self.delta_max_sampling()
        # print(F"Max delta ideal sampling (10 samples per period) (f_0 = {round(f_ten_samples*1e-3)}mm^-1: {round(delta_max_ten_samples * 1e9,2)}nm")
        maxes = [(d_max_field, 'B strength'), (delta_max_env, 'envelope'), (delta_max_ten_samples, 'sampling')]
        (d_max, max_name) = min(maxes)
        # print(min_max)
        return f"""Instrument ID {self.id}
    \tSource: L0 = {self.L0 * 1e10} Å; sigma_L = {self.DL * 1e10} Å
    \tPrecession device: type = {self.prec_type}; theta_0 = {round(self.theta_0,2)} rad; By_min = {self.By_min * 1e3}mT; By_max = {self.By_max * 1e3}mT
    \tdelta range from B strength: {round(d_min * 1e9,2)} - {round(d_max_field * 1e9, 2)}nm
    \tdelta limit for envelope FWHM >= {FWHM_env_min*1e3}mm: {round(delta_max_env * 1e9,2)}nm
    \tmax delta for sampling at 10/s (f_0 = {round(f_s / 10 *1e-3)}mm^-1): {round(delta_max_ten_samples * 1e9,2)}nm
    \tfinal delta range: {round(d_min * 1e9,2)} - {round(d_max * 1e9, 2)}nm ({max_name} limited)"""
    

# print(df.values)
instrs = []
for r in df.values:
    # print(r)
    id = r[0]
    L0 = float(r[1]) * 1e-10
    DL = float(r[2]) * 1e-10 / FWHM_factor
    prec = r[4]
    theta_0 = np.deg2rad(float(r[5]))
    By_min = float(r[6]) * 1e-3
    By_max = float(r[7]) * 1e-3
    instr = Instrument(id, prec, L0, DL, theta_0, By_min, By_max)
    instrs.append(instr)

for instr in instrs:
    print(str(instr))

In [ ]:
N = 1000000
N_steps = 30
def find_overlap(interval1, interval2):
    a1, a2 = interval1
    b1, b2 = interval2

    # Calculate the start and end of the overlap interval
    start = max(a1, b1)
    end = min(a2, b2)

    # Check if there is an actual overlap
    if start <= end:
        return (start, end)
    else:
        return None  # No overlap

def log_overlap_percentage(interval1, interval2):
    a1, a2 = interval1
    b1, b2 = interval2
    log_overlap = find_overlap((np.log(a1), np.log(a2)), (np.log(b1), np.log(b2)))
    if log_overlap == None:
        return 0
    else:
        (a_log, b_log) = log_overlap
        log_fraction = (b_log - a_log) / (np.log(b2) - np.log(b1)) * 100
        return log_fraction 

with open('simulate.sh', 'w') as f:
    f.write('#!/bin/bash\n# Simulation script automatically generated by simulation-drive.ipynb, use this to create variants of it\n')
    f.write('rm -rf data\n')
    for instr in instrs[3:-6]:
        print("-------------------------\n"+str(instr))
        for R in Rs:
            t = thickness_map[(instr.L0 * 1e10, R)]
            mode = 'GPU'
            # if instr.prec_type == 'foil':
            #     mode = 'CPU'
            prec_type = instr.prec_type
            if instr.prec_type == 'wsp':
                prec_type = 'iwsp'
            By = 0.01
            
            d_min, d_max = instr.delta_range()

            # print(f"delta_y range: {round(d_min * 1e9,2)} - {round(d_max * 1e9, 2)}nm")
            # print(f"\tRange of interest for R = {R * 1e9}nm: 0 - {3 * R * 1e9}")
            print(f"\tR = {R * 1e9}nm:")
            print(f"\t\tRange of interest:  {round(0.1 * R * 1e9)} - {round(3 * R * 1e9)}nm")

            

            overlap = find_overlap((d_min, d_max), (0.0, 3 * R))
            if overlap == None:
                print("No overlap!")
                f.write(f'echo "Skipping {instr.id}_{int(R * 1e10)} due to non-overlapping ranges!\n')

                raise Exception("No overlap!")
            else:
                (a,b) = overlap
                fraction = (b - a) / (3 * R) * 100
                log_fraction = log_overlap_percentage((d_min, d_max), (0.1 * R, 3 * R))
                print(f"\t\tOverlap: {round(a * 1e9,2)} - {round(b * 1e9, 2)}nm ({round(fraction,1)}% of linear range, {round(log_fraction,1)}% of log range)")
                max_ratio = b / d_max
                By_min = instr.By_min
                By_max = instr.By_max * max_ratio
                sim_cmd = f"./full-simulation.sh {N} 4 {N_steps} {By_min},{By_max} {instr.L0 * 1e10} {instr.DL* 1e10} {R * 1e10} {t} {prec_type} {prec_type}_empty {mode}; rm -rf data_{instr.id}_{int(R * 1e10)}; mv data data_{instr.id}_{int(R * 1e10)}"
                # print(sim_cmd)
                f.write(f'{sim_cmd}\n')


In [25]:
import pygad

target_interval = (10e-9, 6000e-9)

def optimize_instrument(PG=True):
    # If a PG is used, change permitted L0 range
    if PG:
        quality = 0.01
        gene_space = [{'low': 2.0, 'high': 4.8}, {'low': 1.0, 'high': 10.0}]
        monochrom_name = 'pyroletic graphite'
    else:
        quality = 0.1
        gene_space = [{'low': 7.0, 'high': 12.0}, {'low': 1.0, 'high': 10.0}]
        monochrom_name = 'velocity selector'

    def instrument_from_solution(solution):
        L0 = solution[0] * 1e-10
        L_s = solution[1]
        theta_0 = np.deg2rad(45)
        By_min = 0.1e-3
        By_max = 63e-3
        instr = Instrument('', 'wsp', L0, quality * L0 / FWHM_factor, theta_0, By_min, By_max, L_s)
        return instr

    # Define the fitness function
    def fitness_func(ga_instance, solution, solution_idx):
        instr = instrument_from_solution(solution)
        delta_range = instr.delta_range()
        fitness = log_overlap_percentage(delta_range, target_interval)
        return fitness

    # Number of genes in each solution
    num_genes = 2

        
    # Initialize the GA instance
    ga_instance = pygad.GA(
        mutation_percent_genes = 50,
        num_generations=1000,
        num_parents_mating=5,
        fitness_func=fitness_func,
        sol_per_pop=400,
        num_genes=num_genes,
        suppress_warnings=True,
        parallel_processing=4,
        gene_space=gene_space
    )

    # Run the GA
    ga_instance.run()

    # Retrieve and print the best solution
    best_solution, best_fitness, best_solution_idx = ga_instance.best_solution()
    print(f"Best solution: {best_solution}\nBest fitness: {best_fitness}")
    best_instr = instrument_from_solution(best_solution)

    best_instr.id = f'Best WSP instrument using {monochrom_name}'
    return best_instr

print("==========Best instrument compatible with pyroletic graphite monochromator==========")
pg_instr = optimize_instrument(PG=True)
print(str(pg_instr))

==========Best instrument compatible with pyroletic graphite monochromator==========
Best solution: [4.79975607 2.94533289]
Best fitness: 88.29079317616339
Instrument ID Best WSP instrument using pyroletic graphite
    	Source: L0 = 4.799756073870761 Å; sigma_L = 0.02038268734801635 Å
    	Precession device: type = wsp; theta_0 = 0.79 rad; By_min = 0.1mT; By_max = 63.0mT
    	delta range from B strength: 10.0 - 3150.03nm
    	delta limit for envelope FWHM >= 5.0mm: 24952.79nm
    	max delta for sampling at 10/s (f_0 = 10mm^-1): 14136.88nm
    	final delta range: 10.0 - 3150.03nm (B strength limited)


# Milk sample (from dairy paper)

In [ ]:
delta_rho = 2.0e14 # 1/m^2 (?)
phi = 0.036
R = 50e-9
t = 2e-3
for (L0, _) in sources:
    st = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    print(st)